The sole purpose of this file is to convert the CombinedCSI.mat file from the pipeline to a separate data.npy and mask.npy files for easier usage. You only have to insert the correct data folder name below and run the notebook. A CombinedCSI.mat file in that folder is required

In [1]:
import os
import numpy as np

def normalize_with_reference(folder_list, base_path="."):
    """
    Normiert alle data.npy in folder_list mit demselben Faktor,
    der aus dem Referenz-Ordner (erster in der Liste) kommt.
    
    Args:
        folder_list (list of str): Liste von Ordnernamen, die 'data.npy' enthalten.
        base_path (str): Pfad, unter dem die Ordner liegen.
    """
    if not folder_list:
        raise ValueError("folder_list darf nicht leer sein.")

    # Referenzordner = erster in der Liste
    ref_folder = folder_list[0]
    ref_path   = os.path.join(base_path, ref_folder, "data.npy")
    if not os.path.isfile(ref_path):
        raise FileNotFoundError(f"Referenzdatei nicht gefunden: {ref_path}")

    # Lade Referenzdaten
    ref_data = np.load(ref_path)
    ref_max  = np.max(np.abs(ref_data))
    if ref_max <= 0:
        raise ValueError("Referenz hat keinen positiven Betrag zum Normieren.")
    print(f"[INFO] Referenz {ref_folder}: max(|data|) = {ref_max:.4e}")

    # Normiere alle Ordner
    for fold in folder_list:
        in_path  = os.path.join(base_path, fold, "data.npy")
        out_dir  = os.path.join(base_path, fold + "_normalized")
        out_path = os.path.join(out_dir, "data.npy")

        if not os.path.isfile(in_path):
            print(f"[WARN] Überspringe {fold}, keine data.npy gefunden.")
            continue

        os.makedirs(out_dir, exist_ok=True)

        arr = np.load(in_path)
        arr_norm = arr / ref_max

        np.save(out_path, arr_norm)
        print(f"[OK] {fold} → {out_path}")

    # Speichere den Referenzfaktor in eine Datei
    factor_path = os.path.join(base_path, "normalization_factor.txt")
    with open(factor_path, "w") as f:
        f.write(f"{ref_max:.6e}\n")
    print(f"[INFO] Normierungsfaktor gespeichert in {factor_path}")
# Beispielaufruf:
# folders = ["RefFolder", "Other1", "Other2"]
# normalize_with_reference(folders, base_path="datasets")

In [2]:
folders = ["Simulated_Lac_Random_GT", "Simulated_Lac_Random_1", "Simulated_Lac_Random_2", "Simulated_Lac_Random_3", "Simulated_Lac_Random_4"]
normalize_with_reference(folders)

[INFO] Referenz Simulated_Lac_Random_GT: max(|data|) = 3.8227e+00
[OK] Simulated_Lac_Random_GT → ./Simulated_Lac_Random_GT_normalized/data.npy
[OK] Simulated_Lac_Random_1 → ./Simulated_Lac_Random_1_normalized/data.npy
[OK] Simulated_Lac_Random_2 → ./Simulated_Lac_Random_2_normalized/data.npy
[OK] Simulated_Lac_Random_3 → ./Simulated_Lac_Random_3_normalized/data.npy
[OK] Simulated_Lac_Random_4 → ./Simulated_Lac_Random_4_normalized/data.npy
[INFO] Normierungsfaktor gespeichert in ./normalization_factor.txt


In [2]:
import os
import numpy as np

def normalize_individually(folder_list, base_path="."):
    """
    Normiert jede data.npy in folder_list separat auf max(|data|)=1.
    Speichert Ergebnis in Ordnern mit Suffix '_normalized'.
    Legt in jedem Normalisierungsordner eine 'factor.txt' mit dem Wert ab.
    
    Args:
        folder_list (list of str): Liste von Ordnernamen, die 'data.npy' enthalten.
        base_path (str): Pfad, unter dem die Ordner liegen.
    """
    for fold in folder_list:
        in_path  = os.path.join(base_path, fold, "data.npy")
        out_dir  = os.path.join(base_path, fold + "_normalized")
        out_path = os.path.join(out_dir, "data.npy")

        if not os.path.isfile(in_path):
            print(f"[WARN] Überspringe {fold}, keine data.npy gefunden.")
            continue

        os.makedirs(out_dir, exist_ok=True)

        arr = np.load(in_path)
        maxv = np.max(np.abs(arr))
        if maxv <= 0:
            print(f"[WARN] {fold}: max(|data|) <= 0, Normierung übersprungen.")
            np.save(out_path, arr)
            continue

        arr_norm = arr / maxv
        np.save(out_path, arr_norm)

        # Speichere den Faktor in eine Datei im Normalisierungsordner
        factor_path = os.path.join(out_dir, "factor.txt")
        with open(factor_path, "w") as f:
            f.write(f"{maxv:.6e}\n")

        print(f"[OK] {fold}: normiert mit Faktor {maxv:.4e} → {out_path}")

In [4]:
folders = ["P03", "P04", "P05", "P06", "P07", "P08"]
normalize_individually(folders)

[OK] P03: normiert mit Faktor 4.1643e+04 → ./P03_normalized/data.npy
[OK] P04: normiert mit Faktor 4.8763e+04 → ./P04_normalized/data.npy
[OK] P05: normiert mit Faktor 4.2078e+04 → ./P05_normalized/data.npy
[OK] P06: normiert mit Faktor 5.5208e+04 → ./P06_normalized/data.npy
[OK] P07: normiert mit Faktor 4.5706e+04 → ./P07_normalized/data.npy
[OK] P08: normiert mit Faktor 4.5392e+04 → ./P08_normalized/data.npy


In [4]:
from scipy.io import loadmat

data = loadmat("Simulated_Lesion_GT/Lesion_120pts.mat")

In [1]:
import numpy as np
import nibabel as nib

lac_img = nib.load("Lac_amp_map.nii")   # oder .nii
glx_img = nib.load("Glx_amp_map.nii")

lac = lac_img.get_fdata(dtype=np.float32)
glx = glx_img.get_fdata(dtype=np.float32)

# sichere Division (vermeidet /0)
ratio = np.divide(lac, glx, out=np.zeros_like(lac, dtype=np.float32), where=glx!=0)

# optional: ungültige/negative Werte maskieren
ratio[~np.isfinite(ratio)] = 0
ratio[glx <= 0] = 0

ratio_img = nib.Nifti1Image(ratio.astype(np.float32), lac_img.affine, lac_img.header)
nib.save(ratio_img, "Lac_over_Glx_ratio.nii")